In [159]:
%matplotlib inline
from __future__ import division
import numpy as np
from scipy.stats import chi2
import matplotlib.pyplot as plt
import cProfile
from lxml import html
import requests
import unicodedata
import re, htmlentitydefs
from collections import defaultdict
import json
import pickle

In [3]:
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub) # use start += 1 to find overlapping matches

In [11]:
def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

print np.unique(region)

print [strip_accents(t) for t in np.unique(region)]

[u'Aguascalientes' u'Baja California Sur' u'Campeche' u'Chiapas'
 u'Coahuila' u'Colima' u'Durango' u'Edo. de M\xe9xico' u'Guanajuato'
 u'Jalisco' u'Michoac\xe1n' u'Morelos' u'Nayarit' u'Nuevo Le\xf3n'
 u'Oaxaca' u'Quer\xe9taro' u'San Luis Potos\xed' u'Sinaloa' u'Sonora'
 u'Tlaxcala' u'Veracruz' u'Yucat\xe1n' u'Zacatecas']
[u'Aguascalientes', u'Baja California Sur', u'Campeche', u'Chiapas', u'Coahuila', u'Colima', u'Durango', u'Edo. de Mexico', u'Guanajuato', u'Jalisco', u'Michoacan', u'Morelos', u'Nayarit', u'Nuevo Leon', u'Oaxaca', u'Queretaro', u'San Luis Potosi', u'Sinaloa', u'Sonora', u'Tlaxcala', u'Veracruz', u'Yucatan', u'Zacatecas']


In [50]:
www = 'http://www.economia-sniim.gob.mx/SNIIM-Pecuarios-Nacionales/e_Pie.asp?origen=0&destino=0&del=29&al=29&mes=02&anio=2016&RegPag=1000&x=27&y=12&Var=Bov'

page = requests.get(www)
tree = html.fromstring(page.content)

## display html
print html.tostring(tree, pretty_print=True)

<html>

<head>
<meta name="GENERATOR" content="Microsoft FrontPage 5.0">
<title>
Bovinos : Ganado en Pie</title>
<link rel="StyleSheet" href="/estilo.css" type="text/css">
</head>
<body bgcolor="#FFFFFF" text="#000000" link="#003399" vlink="#663366" alink="#8888AA" topmargin="8" leftmargin="7">

<div align="center"><center>
    <table align="center" border="0">
      <tr> 
        <td align="center"> <img src="/titulo.gif" width="427" height="19">
</td>
        <td><img src="/logo-sniim.gif" border="0" align="right" width="88" height="43"></td>
      </tr>
    </table>
    <table width="103%" border="0" cellpadding="2" cellspacing="1" bordercolor="#CCCCCC">
      <form action="lucas3.asp" method="POST">
        <tr> 
          <td width="100%" class="encabezado"> Mercados Nacionales Pecuarios</td>
        </tr>
        
      </form>
    </table>
    <table align="center" border="0">
      <tr> 
        <td class="encabPRE"> Precios de 
          Bovinos : Ganado en Pie

</td>
      </

In [25]:
region = tree.xpath('//td[@class="Datos"]/text()')
prices = tree.xpath('//td[@class="DatosNum"]/text()')
markets = tree.xpath('//td[@class="encabDES"]/text()')
extra_info = tree.xpath('//td[@class="encabFEC"]/text()')

ratio = int(len(prices) / len(region))
print " # regions: ", len(region), " | # prices: ", len(prices), " | prices / region = ", ratio

 # regions:  46  | # prices:  276  | prices / region =  6


In [65]:
for elt in tree:
    print len(elt), elt.tag
    for sub_elt in elt:
        print "--", len(sub_elt), sub_elt.tag, sub_elt.attrib

3 head
-- 0 meta {'content': 'Microsoft FrontPage 5.0', 'name': 'GENERATOR'}
-- 0 title {}
-- 0 link {'href': '/estilo.css', 'type': 'text/css', 'rel': 'StyleSheet'}
6 body
-- 1 div {'align': 'center'}
-- 1 div {'align': 'center'}
-- 1 table {'cellpadding': '0', 'cellspacing': '0', 'border': '0', 'width': '100%'}
-- 1 div {'align': 'center'}
-- 0 p {'align': 'center'}
-- 1 div {'align': 'center'}


In [131]:
def extract_info_tree(tree, market_name='encabDES', region_name='Datos', price_name='DatosNum'):

    all_elt = [tree] ; to_be_read = [tree] ; t_markets = []
    t_markets_regions = defaultdict(list) ; t_markets_prices = defaultdict(list)
    
    current_market = -1

    while len(to_be_read) > 0:
        elt = to_be_read[0]
        del to_be_read[0]

        for sub_elt in elt:
            all_elt.append(sub_elt)
            to_be_read.append(sub_elt)

            if 'class' in sub_elt.attrib and sub_elt.tag == "td":
                if sub_elt.attrib['class'] == market_name:
                    # next market was found
                    t_markets.append(sub_elt)
                    current_market = sub_elt.text
                if sub_elt.attrib['class'] == region_name:
                    # region for current market
                    t_markets_regions[current_market].append(sub_elt.text)
                if sub_elt.attrib['class'] == price_name:
                    # price for current (market, region)
                    t_markets_prices[current_market].append(sub_elt.text)
                    
    return t_markets, t_markets_regions, t_markets_prices

def combine_data(t_markets_regions, t_markets_prices):
    
    t_data = defaultdict(list)
    
    for k in t_markets_regions.keys():
        
        regions = t_markets_regions[k]
        prices = t_markets_prices[k]
        ratio = int(len(prices) / len(regions))
    
        for i, r in enumerate(regions):
            p_r = prices[i*ratio:(i+1)*ratio]
            t_data[k].append((r, p_r))

    return t_data

In [106]:
t_markets, t_markets_regions, t_markets_prices = extract_info_tree(tree)
t_data = combine_data(t_markets_regions, t_markets_prices)

In [107]:
for key, val in t_data.iteritems():
    print key, val

Ver. : Rastro Municipal de Coatepec [('Veracruz', ['3', '503', '28.00', '-', '-', '-'])]
Coah. : TIF 40 Procarne [('Coahuila', ['313', '370', '40.00', '-', '-', '-'])]
Ags. : Frigorífico y Empacadora de Aguascalientes [('Aguascalientes', ['137', '564', '-', '-', '38.00', '58.00']), ('Jalisco', ['29', '565', '-', '-', '38.00', '58.00']), ('Zacatecas', ['8', '550', '-', '-', '38.00', '-'])]
Coah. : R.M. Torreón [('Coahuila', ['67', '485', '45.00', '-', '28.00', '-'])]
Edo. Méx. : Tlalnepantla [('Aguascalientes', ['20', '500', '47.00', '-', '35.00', '-']), (u'Edo. de M\xe9xico', ['8', '498', '47.00', '-', '35.00', '-']), (u'Quer\xe9taro', ['31', '500', '47.00', '-', '35.00', '-'])]
N.L. : Procesadora Selecta, S.A. de C.V. [(u'Nuevo Le\xf3n', ['25', '450', '44.50', '-', '-', '-'])]
Jal. : R.M. Tlaquepaque [('Jalisco', ['31', '450', '46.50', '-', '33.50', '-'])]
Oax. : Rastro Reforma Agraria [('Oaxaca', ['41', '-', '42.00', '-', '35.00', '-'])]
Gto. : R.P. León, Serv. Integral [('Guanajuato

In [126]:
def display_data(t_data):
    i = 0
    for k, v in t_data.iteritems():
        print i, ") Rastro : ", k
        print "------------------------------------------------------------"
        for elt in v:
            print "Estado: ", elt[0], " || Precios: ", elt[1]
        print "------------------------------------------------------------"
        print ""
        i += 1

In [153]:
def read_data_day(day=1, month=1, year=2016):
    www = 'http://www.economia-sniim.gob.mx/SNIIM-Pecuarios-Nacionales/e_Pie.asp?origen=0&destino=0&del={}&al={}&mes={}&anio={}&RegPag=1000&x=27&y=12&Var=Bov'.format(day, day, month, year)
    page = requests.get(www)
    tree = html.fromstring(page.content)
    t_markets, t_markets_regions, t_markets_prices = extract_info_tree(tree)
    t_data = combine_data(t_markets_regions, t_markets_prices)
    return t_data, t_markets_regions, t_markets_prices


def read_data_month(month=1, year=2016, initial_day=1, final_day=31):
    t_data_month = defaultdict(list)
    for day in range(initial_day, final_day+1):
        d_day, t_markets_regions, t_markets_prices = read_data_day(day, month, year)
        if len(d_day.keys()) > 0:
            t_data_month[(day, month, year)] = d_day
    return t_data_month


def read_data_year(year=2016, initial_month=1, final_month=12):
    t_data_year = defaultdict(list)
    for month in range(initial_month, final_month+1):
        print "Reading Month ", month, "..."
        t_data_month = read_data_month(month, year)
        if len(t_data_month.keys()) > 0:
            t_data_year[(month, year)] = t_data_month
    return t_data_year

In [129]:
concepts = ["numero cabezas", "peso promedio", "precio novillo", "precio novillona", "precio vaca", "precio toro"]
# prices correspond to those concepts

In [132]:
t_data, t_markets_regions, t_markets_prices = read_data_day(day=35, month=2, year=2016)
display_data(t_data)

In [150]:
t_data_month = read_data_month(month=1, year=2016)

In [151]:
for k, val in sorted(t_data_month.iteritems()):
    print k, len(val)

(3, 1, 2016) 1
(4, 1, 2016) 30
(5, 1, 2016) 31
(6, 1, 2016) 31
(7, 1, 2016) 29
(8, 1, 2016) 30
(9, 1, 2016) 3
(10, 1, 2016) 1
(11, 1, 2016) 33
(12, 1, 2016) 31
(13, 1, 2016) 32
(14, 1, 2016) 29
(15, 1, 2016) 30
(16, 1, 2016) 2
(17, 1, 2016) 1
(18, 1, 2016) 33
(19, 1, 2016) 33
(20, 1, 2016) 32
(21, 1, 2016) 28
(22, 1, 2016) 30
(23, 1, 2016) 2
(24, 1, 2016) 1
(25, 1, 2016) 34
(26, 1, 2016) 32
(27, 1, 2016) 32
(28, 1, 2016) 29
(29, 1, 2016) 24
(30, 1, 2016) 3
(31, 1, 2016) 1


In [155]:
t_data_year = read_data_year(year=2015)

Reading Month  1 ...
Reading Month  2 ...
Reading Month  3 ...
Reading Month  4 ...
Reading Month  5 ...
Reading Month  6 ...
Reading Month  7 ...
Reading Month  8 ...
Reading Month  9 ...
Reading Month  10 ...
Reading Month  11 ...
Reading Month  12 ...


In [156]:
# save the data
np.save('meat_2015_mexico.npy', t_data_year)

In [160]:
# save
with open('meat_2015_mexico.txt', 'wb') as handle:
    pickle.dump(t_data_year, handle)

In [161]:
# read
with open('meat_2015_mexico.txt', 'rb') as handle:
    data = pickle.loads(handle.read())    

In [163]:
data.keys()

[(10, 2015),
 (5, 2015),
 (2, 2015),
 (11, 2015),
 (6, 2015),
 (8, 2015),
 (3, 2015),
 (12, 2015),
 (7, 2015),
 (9, 2015),
 (4, 2015),
 (1, 2015)]

In [128]:
##
# Removes HTML or XML character references and entities from a text string.
#
# @param text The HTML (or XML) source text.
# @return The plain text, as a Unicode string, if necessary.

def unescape(text):
    def fixup(m):
        text = m.group(0)
        if text[:2] == "&#":
            # character reference
            try:
                if text[:3] == "&#x":
                    return unichr(int(text[3:-1], 16))
                else:
                    return unichr(int(text[2:-1]))
            except ValueError:
                pass
        else:
            # named entity
            try:
                text = unichr(htmlentitydefs.name2codepoint[text[1:-1]])
            except KeyError:
                pass
        return text # leave as is
    return re.sub("&#?\w+;", fixup, text)